# Install packages

# General setup

In [1]:
import os # importing OS in order to make GPU visible
import time
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]='1'

In [2]:
#code structure below courtesy https://github.com/nicknochnack/TFODCourse
MODEL_NAME = 'resnet50' 
MODEL_VERSION = 'v4'
PRETRAINED_MODEL_NAME = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz'
CLASS_LABELS_FILE = 'Class Labels.txt'
LABEL_MAP_NAME = 'label_map.pbtxt'
TFR_FILENAME = 'cleaned_complete'
TEST_RATIO = .2
TRAIN_RECORD = 'xview_train_cleaned_complete.record'
EVAL_RECORD = 'xview_test_cleaned_complete.record'
TEST_RECORD = 'xview_test_cleaned.record'
GEOJSON_FILE = 'train_data_clean.geojson'

XVIEW_UTILS_PATH = r'C:\Users\bcper\Documents\GitHub'
TF_INSTALL_PATH = r'C:\Users\bcper\Documents\GitHub\models'

NUM_TRAINING_IMAGES = 44460
NUM_EPOCHS = 10
BATCH_SIZE = 6

STEPS = int(NUM_TRAINING_IMAGES / BATCH_SIZE * NUM_EPOCHS)


In [3]:
DATA_DIR_PATH = os.path.join(TF_INSTALL_PATH, 'workspace','data', '')

paths = {
    'WORK_PATH': os.path.join(TF_INSTALL_PATH, 'workspace'),
    'TFR_WORK_PATH': os.path.join(XVIEW_UTILS_PATH,'xView_data_utilities'),
    'SCRIPTS_PATH': os.path.join(TF_INSTALL_PATH,'research', 'object_detection'),
    'LABELMAP_PATH': DATA_DIR_PATH,
    'TFR_TRAIN_IMAGE': os.path.join(DATA_DIR_PATH, 'train_images',''),
    'TFR_TEST_IMAGE': os.path.join(DATA_DIR_PATH, 'test_images', ''),
    'MODEL_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','models', MODEL_NAME, MODEL_VERSION, ''),
    'PRETRAINED_MODEL': os.path.join(TF_INSTALL_PATH, 'workspace','pre-trained_models'),
    'CHECKPOINT_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','models', MODEL_NAME), 
    'EXPORT_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','exported_models',MODEL_NAME + '_' + MODEL_VERSION),
    'PROTOC_PATH':os.path.join(TF_INSTALL_PATH,'protoc'),
    'GEOJSON_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','data'),
    'CLASS_LABEL_PATH': DATA_DIR_PATH,
    'DATA_PATH': DATA_DIR_PATH
 }

In [4]:
files = {
    'PIPELINE_CONFIG_SOURCE': os.path.join(paths['PRETRAINED_MODEL'],PRETRAINED_MODEL_NAME, 'pipeline.config'), 
    'PIPELINE_CONFIG':os.path.join(TF_INSTALL_PATH, 'workspace','models', MODEL_NAME, MODEL_VERSION, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['LABELMAP_PATH'], LABEL_MAP_NAME),
    'GEOJSON': os.path.join(paths['GEOJSON_PATH'], GEOJSON_FILE),
    'CLASS_LABELS': os.path.join(paths['CLASS_LABEL_PATH'], CLASS_LABELS_FILE),
    'TRAIN_RECORD': os.path.join(paths['DATA_PATH'], TRAIN_RECORD),
    'EVAL_RECORD': os.path.join(paths['DATA_PATH'], EVAL_RECORD),
    'TEST_RECORD': os.path.join(paths['DATA_PATH'], TEST_RECORD),
    'FINE_TUNE_CKPT': os.path.join(paths['PRETRAINED_MODEL'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0'),
    'TRAINING_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'model_main_tf2.py')
}

In [5]:
config = {
    'CLASSES': 95,
    'STEPS': STEPS,
    'BATCH_SIZE': BATCH_SIZE,
    'WARMUP_LEARN_RATE': .001,
    'BASE_LEARN_RATE': .01 ,
    'WARMUP_STEPS': 1000
}

In [6]:
train_model = '{} --pipeline_config_path={} --model_dir={}  --alsologtostderr'.format('model_main_tf2.py', files['PIPELINE_CONFIG'], paths['MODEL_PATH'])
print('Model Training command: \n')
print(train_model)
print()
evaluate_model = '{} --pipeline_config_path={} --model_dir={} --checkpoint_dir={}  --sample_1_of_n_eval_examples=1'.format('model_main_tf2.py', files['PIPELINE_CONFIG'], paths['MODEL_PATH'], paths['MODEL_PATH'])
print('Model Evaluation Command: \n')
print(evaluate_model)
print()
run_tensorboard = 'tensorboard --logdir={}'.format(paths['CHECKPOINT_PATH'])
print('Tensorboard Command: \n')
print(run_tensorboard)
print()

Model Training command: 

model_main_tf2.py --pipeline_config_path=C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50\v4\pipeline.config --model_dir=C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50\v4\  --alsologtostderr

Model Evaluation Command: 

model_main_tf2.py --pipeline_config_path=C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50\v4\pipeline.config --model_dir=C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50\v4\ --checkpoint_dir=C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50\v4\  --sample_1_of_n_eval_examples=1

Tensorboard Command: 

tensorboard --logdir=C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50



# Generate the TF .record files

In [13]:
os.chdir(paths['TFR_WORK_PATH'])
TFR_TEST_FILENAME = TFR_FILENAME + 'test'


generate_train_and_eval = 'process_wv.py {} {} -s={} -t={}'.format(paths['TFR_TRAIN_IMAGE'], files['GEOJSON'], TFR_FILENAME, TEST_RATIO)
%run {generate_train_and_eval}

#generate_test = 'process_wv.py {} {} -s={} -t={}'.format(paths['TFR_TEST_IMAGE'], files['GEOJSON'], TFR_FILENAME, 0)
#%run {generate_test}

100%|██████████████████████████████████████████████████████████████████████| 468474/468474 [00:01<00:00, 268466.53it/s]
INFO:root:Res: (500, 500)
100%|████████████████████████████████████████████████████████████████████████████████| 845/845 [09:03<00:00,  1.56it/s]
INFO:root:Max chips per resolution: 21139 
INFO:root:Tot Box: 462529
INFO:root:Chips: 14093
INFO:root:Res: (400, 400)
100%|████████████████████████████████████████████████████████████████████████████████| 845/845 [09:18<00:00,  1.51it/s]
INFO:root:Tot Box: 498253
INFO:root:Chips: 20748
INFO:root:Res: (300, 300)
100%|████████████████████████████████████████████████████████████████████████████████| 845/845 [09:39<00:00,  1.46it/s]
INFO:root:Tot Box: 360212
INFO:root:Chips: 21139
INFO:root:saved: 11520 train chips
INFO:root:saved: 44460 test chips


# Create the label map

In [ ]:
os.chdir(paths['DATA_PATH'])
#read in the class labels
class_dict = {}
file = open(files['CLASS_LABELS'], "r")

#create a class label dictionary
for line in file:
    key, value = line.split(':')
    class_dict[int(key)] = value.strip()


#this will make the list in the correct label format for tensorflow
x = ''
for key in class_dict:
    x = x + ('item { \n\tid: ' + str(key) + '\n\tname: \"' + class_dict.get(key) + '\"\n}\n\n')
    
# Write the label map to a file
lmap = open(files['LABELMAP'],"w")#write mode
lmap.write(x)
lmap.close()

# When creating a new model using existing data created above

# Update the .config file

In [7]:
import re
import io
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

config_file = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG_SOURCE'])

In [8]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG_SOURCE'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

    
#this is specific to the faster_rcnn model, for others we would need to change this
pipeline_config.model.faster_rcnn.num_classes = config['CLASSES']



pipeline_config.train_config.batch_size = config['BATCH_SIZE']
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(files['FINE_TUNE_CKPT'])
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_config.num_steps = config['STEPS']
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [files['TRAIN_RECORD']]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [files['EVAL_RECORD']]
pipeline_config.train_config.use_bfloat16 = False
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = config['STEPS']
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = config['WARMUP_LEARN_RATE']
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = config['BASE_LEARN_RATE']
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_steps = config['WARMUP_STEPS']


config_text = text_format.MessageToString(pipeline_config)    

#create the folder if if doesnt exist
if not os.path.exists(paths['MODEL_PATH']):
    os.makedirs(paths['MODEL_PATH'])
   
    
if os.path.exists(files['PIPELINE_CONFIG']):
    with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
        f.write(config_text)
else:
    open(paths['MODEL_PATH'] + "pipeline.config", "x")
    
    with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
        f.write(config_text)



## Start the model training

In [7]:
os.chdir(paths['SCRIPTS_PATH'])
print('Training model for {} steps.'.format(config['STEPS']))

print('Starting at time: ' + str(time.strftime("%I:%M:%S")))
start_time = time.time()

#run the model training
%run {train_model}

end_time = time.time()
print("Train time: " + str(round(end_time - start_time, 2)) + " seconds")

Training model for 74100 steps.
Starting at time: 08:20:53


W0610 08:20:57.794448 12508 mirrored_strategy.py:379] Collective ops is not configured at program startup. Some performance features may not be enabled.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


I0610 08:20:58.440447 12508 mirrored_strategy.py:369] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Maybe overwriting train_steps: None


...

INFO:tensorflow:Step 74000 per-step time 0.351s loss=0.745


I0610 10:54:31.067357 12508 model_lib_v2.py:680] Step 74000 per-step time 0.351s loss=0.745


INFO:tensorflow:Step 74100 per-step time 0.368s loss=0.931


I0610 10:55:07.849004 12508 model_lib_v2.py:680] Step 74100 per-step time 0.368s loss=0.931


Train time: 9255.14 seconds


## Start the model evaluation

In [ ]:
os.chdir(paths['SCRIPTS_PATH'])

#run the model evaluation
%run {evaluate_model}

W0610 10:57:50.221807 10764 model_lib_v2.py:1063] Forced number of epochs for all eval validations to be 1.


INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1


I0610 10:57:50.222807 10764 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1


INFO:tensorflow:Maybe overwriting use_bfloat16: False


I0610 10:57:50.223807 10764 config_util.py:552] Maybe overwriting use_bfloat16: False


...

I0610 11:34:41.641754 10764 model_lib_v2.py:992] 	+ Loss/regularization_loss: 0.000000


INFO:tensorflow:	+ Loss/total_loss: 1.290246


I0610 11:34:41.643754 10764 model_lib_v2.py:992] 	+ Loss/total_loss: 1.290246


INFO:tensorflow:Waiting for new checkpoint at C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50\v4\


I0610 11:34:43.107737 10764 checkpoint_utils.py:140] Waiting for new checkpoint at C:\Users\bcper\Documents\GitHub\models\workspace\models\resnet50\v4\


## Start Tensorboard

In [9]:
os.chdir(paths['WORK_PATH'])
tensorboard_path = paths['CHECKPOINT_PATH']

!tensorboard --logdir=$tensorboard_path

^C


NameError: name 'tensorboard' is not defined

# Export Model

In [ ]:
os.chdir(paths['SCRIPTS_PATH'])

export_model = '{} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={} --input_type=image_tensor'.format('exporter_main_v2.py', files['PIPELINE_CONFIG'], paths['MODEL_PATH'], paths['EXPORT_PATH'])
%run {export_model}

print(export_model)
